![alt text](img/MIoT_ML.png "MIoT_ML")
# Unidad 04  Entrenamiento de Modelos de Aprendizaje Automático

El objetivo principal de esta práctica es el conocer los pasos básicos para el desarrollo, optimización y evaluación de los modelos de Aprendizaje Automático. 
El Notebook contiene varios ejercicios sencillos. Debéis desarrollarlos durante la clase y subirlos al aula virtual.

## Referencias útiles para la práctica
1. API Pandas: [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)
2. API Scikit-learn: [https://scikit-learn.org/stable/api/index.html](https://scikit-learn.org/stable/api/index.html)
3. Dataset para el ejercicio: [https://www.kaggle.com/datasets/camnugent/california-housing-prices](https://www.kaggle.com/datasets/camnugent/california-housing-prices)
4. Géron, Aurélien. Hands-on machine learning with Scikit-Learn, Keras, and TensorFlow. " O'Reilly Media, Inc.", 2022.
5. Bergstra, J., & Bengio, Y. (2012). Random search for hyperparameter optimization. Journal of machine learning research, 13 (2). *Para profundizar en la optimización de hiperparámetros*
6. Girish Chandrashekar and Ferat Sahin. A survey on feature selection methods. Computers & Electrical Engineering, 40(1):16–28, 2014
   
## 1. Flujo de trabajo básico en problemas de Aprendizaje Automático (*ML workflow*)
A la hora de enfrentarnos a un nuevo problema de Aprendizaje Automático (ML), existen una serie de pasos típicos y comunes que debemos afrontar:
1. Entender el problema y su contexto.
2. Obtener los datos (histórico).
3. Explorar, analizar y entender los datos.
4. Preparar los datos para los modelos.
5. Seleccionar, optimizar y entrenar los modelos ML.
6. Evaluar y presentar el modelo seleccionado.
7. Desplegar, monitorizar y mantener la solución.

En esta unidad nos centraremos en los pasos 5 y 6 del flujo de trabajo.

## 2. Carga y partición de datos
### 2.1. Carga de los datos
Cargaremos los datos generados durante la Unidad02, almacenados al finalizar la misma. Suponemos que se han mantenido los nombres generados durante dicha unidad, y que los datos están almacenados en el directorio raíz.

- Características de entrada preprocesadas (*Inputs*) de las observaciones del conjunto de entrenamiento (*preprocessing_trainset_inputs.csv*).
- Etiquetas de salida (*Outputs*) de las observaciones del conjunto de entrenamiento (*trainset_ouputs.csv*).
- Características de entrada preprocesadas (*Inputs*) de las observaciones del conjunto de Test (*preprocessing_testset_inputs.csv*).
- Etiquetas de salida (*Outputs*) de las observaciones del conjunto de Test (*testset_outputs.csv*).
- Columna de los índices "*idx*" (tal como se guardó en la Unidad 02).

In [ ]:
# pandas library
import pandas as pd
try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd

# leemos los datos de training, indicando que la columna que tiene los índices originales es "idx", que se mantendrá en los nuevos Dataframes
training_inputs = pd.read_csv("./preprocessing_trainset_inputs.csv", index_col="idx")
training_outputs = pd.read_csv("./trainset_ouputs.csv", index_col="idx")
print("Longitud del conjunto de entrenamiento:", len(training_inputs))

# leemos los datos de testing
testing_inputs = pd.read_csv("./preprocessing_testset_inputs.csv", index_col="idx")
testing_outputs = pd.read_csv("./testset_outputs.csv", index_col="idx")
print("Longitud del conjunto de test:", len(testing_inputs))

### 2.2. Creación del conjunto de validación

Si tenemos solo 2 conjuntos para entrenar y para testear, no podemos comparar modelos ni configuraciones de hiperparámetros. **Un error común** es probar o comparar los resultados conseguidos por un modelo o configuración contra el conjunto de Test. **Esto es un error grave**. 

El conjunto de Test no puede emplearse hasta que todas las decisiones estén ya tomadas y el modelo desarrollado. El conjunto de Test es el que usaremos para obtener las métricas generales, y que consideramos representativas del comportamiento del modelo en el futuro con datos nunca vistos. Si se toma cualquier decisión basada en los resultados de Test, estamos *contaminando* el conjunto y las métricas finales serán excesivamente optimistas, ya que están *mejoradas* al tomar una decisión que las hizo aumentar (no podéis esperar que, al desplegar el modelo, los datos nunca vistos tengan las mismas métricas.


Para poder comparar modelos y/o configuraciones, tenemos 2 aproximaciones:
1. Generamos un conjunto de validación y lo empleamos para las comparaciones.
2. Empleamos la validación cruzada con el conjunto de entrenamiento.


Con una de las 2 aproximaciones ya sería suficiente pero, en esta unidad, por motivos docentes, veremos las 2 alternativas en diferentes aspectos.


A continuación vamos a **crear un conjunto de validación**. En las unidades anteriores reservamos un 30% de los datos para evaluar y validar el modelo. La distribución típica de los datos al trabajar con 3 conjuntos (train, val y test) es 70%, 15% y 15% repectivamente, por lo que usaremos el 30% reservado previamente para generar 2 subconjuntos que representen el 15% cada uno del total. El 15% del conjunto de Test lo reservaremos hasta el final.

In [ ]:
try:
    from sklearn.model_selection import train_test_split
except ImportError as err:
    !pip install sklearn
    from sklearn.model_selection import train_test_split

SEED=1234

# dividimos el conjunto de test original: testing_inputs
test_inputs, val_inputs = train_test_split(testing_inputs, test_size=0.5, train_size=0.5, random_state=SEED, shuffle=True)

# generamos los outputs correspondientes a los dos conjuntos creados
# val_inputs.index es la lista de índices seleccionados para el conjunto de inputs de validación
# deben ser los mismos índices para el conjunto de outputs de validación
val_outputs = testing_outputs.loc[val_inputs.index]
print("Longitud del conjunto de validación:", len(val_outputs))

# igualmente para el conjunto de test
test_outputs = testing_outputs.loc[test_inputs.index]
print("Longitud del conjunto de test:", len(test_outputs))

## 3. Creación y evaluación de un modelo básico

### 3.1. Selección de una métrica de evaluación

Para medir la calidad del modelo que vamos a desarrollar, necesitamos seleccionar una métrica que nos dé un valor que cuantifique la diferencia entre los datos conocidos del conjunto de test y los datos predichos por el modelo. 
Una medida de rendimiento típica para los problemas de regresión es el error cuadrático medio (RMSE) que nos da una idea de cuánto error suele cometer el sistema en sus predicciones en las mismas unidades que la variable dependiente, lo que puede facilitar la interpretación de los resultados. 

$$RMSE=\large\sqrt{\frac{1}{n} \sum_{i=1}^N{ (\hat{y}_i-y_i)^2}}$$



El RMSE tiene una serie de características que debemos tener en cuenta a la hora de seleccionarlo como, por ejemplo: 

* Promedia los errores individuales sin tener en cuenta el signo.
    * No informa de los sesgos sistemáticos (ej. el modelo tiende a sobrestimar o subestimar).
* Penaliza los errores grandes.
* Muy sensible a los atípicos (*outliers*).
    * Pueden disparar la métricas y dar una percepción inexacta.
 
Existen otras métricas interesantes para regresión que debemos valorar dependiendo de nuestro objetivo y nuestro problema. Por ejemplo, si el número de *outliers* en nuestras manzanas fuese muy alto, quizás nos conveniese emplear una métrica robusta contra los atípicos como el error absoluto medio (MAE):

$$\large MAE=\frac{1}{N}\sum_{i=1}^N|y_i-\hat{y}_i|$$



En nuestro caso, vamos a emplear la métrica RMSE (en Scikit-learn está disponible mediante la función "[*root_mean_squared_error*](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html)").

In [ ]:
from sklearn.metrics import root_mean_squared_error


### 3.2. Creación y evaluación de un modelo básico
#### 3.2.1. Creación de un modelo de regresión lineal

Como dijimos en la unidad anterior, en la vida real normalmente no programaremos un modelo, sino que eligiremos uno de los muchos que hay en las librerías conocidas, como Scikit-learn. En este caso, vamos a elegir un modelo básico de regresión lineal multivariable (varios atributos de entrada), que en Scitkit-learn es implementado por la clase **LinearRegression**,  y llamaremos a su función *fit* pasándole las entradas y los valores reales (del conjunto de entrenamiento) para que entrene el modelo con esos datos y ajuste sus parámetros. El objetivo final es obtener un modelo que sea capaz de predecir sobre datos nunca vistos.

**Nota**: el entrenamiento de los modelos se puede añadir pipeline que generamos con las operaciones de preprocesado.

In [ ]:
from sklearn.linear_model import LinearRegression

# Creamos un objeto de la clase LinearRegression
model_lin_reg = LinearRegression()
# Ajustamos (entrenamos) el modelo en base a los datos de entrenamniento (inputs y outputs)
model_lin_reg.fit(training_inputs, training_outputs)


#### 3.2.2. Evaluación con el dataset de entrenamiento

Una vez entrenado el modelo con el conjunto de entrenamiento, podemos ver qué tal es su desempeño sobre el conjunto de entrenamiento (midiéndolo con la métrica RMSE). El grado de acierto sobre las primeras muestras:


In [ ]:

# El método predict permite generar predicciones sobre los Inputs pasados como argumento
lr_training_predictions = model_lin_reg.predict(training_inputs)

print("Primeras 5 observaciones del entrenamiento")
for pred,real in zip(lr_training_predictions[:5].round(-2) ,training_outputs.iloc[:5].values):
    print(f"Valor real: {real}, valor predicho: {pred}. Diferencia en absoluto{abs(real-pred)}. Porcentaje desvío: {(abs(real-pred)*100/real).round(2)}%")


Y el grado de acierto sobre todo el dataset de entrenamiento.

In [ ]:
try:
    import matplotlib.pyplot as plt
except ImportError as err:
    !pip install matplotlib
    import matplotlib.pyplot as plt

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("Training score:", model_lin_reg.score(training_inputs, training_outputs))

# Calculamos el RMSE sobre todo el conjunto de datos de entrenamiento comparando la realidad con la predicción
lr_rmse_train = root_mean_squared_error(training_outputs, lr_training_predictions)
print(f"RMSE del conjunto de entrenamiento con modelo de regresión lineal: {lr_rmse_train:.2f}")

plt.scatter(training_outputs, lr_training_predictions)
plt.xlabel("Realidad")
plt.ylabel("Predicción")
plt.title("Regresión lineal - Conjunto de entrenamiento")
plt.show()



En este caso, los resultados no son muy buenos, y todo indica que este problema no puede modelarse con un modelo tan simple y que requiere de alternativas más complejas.  De cualquier forma, es importante recalcar que las métricas sobre el conjunto de entrenamiento son poco relevantes, ya que son los datos empleados para generar el propio modelo y, típicamente, suelen ser métricas muy optimistas y no pueden considerarse métricas generales. Se considera que el modelo está sobreentrenado con esos datos. Esto lo veréis más claro con el ejemplo siguiente (árbol de decisión binaria). Para obtener métricas de comportamiento general, es necesario evaluar el modelo contra datos nunca vistos. 

#### 3.2.3. Evaluación con un conjunto de datos no visto previamente

Veamos ahora cómo se comporta el modelo con un conjunto de datos que no ha visto durante el entrenamiento. Usaremos para esto el conjunto de validación. Recordad que el conjunto de Test está reservado y no lo emplearemos hasta el final.

In [ ]:
# Evaluación con el dataset de validación para el modelo de regresión lineal

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("Val score:", model_lin_reg.score(val_inputs, val_outputs))

lr_val_predictions = model_lin_reg.predict(val_inputs)
lr_rmse_val = root_mean_squared_error(val_outputs, lr_val_predictions)
print(f"RMSE del conjunto de validación con modelo de regresión lineal: {lr_rmse_val:.2f}")

plt.scatter(val_outputs, lr_val_predictions)
plt.xlabel("Realidad")
plt.ylabel("Predicción")
plt.title("Regresión lineal - Conjunto de validación")
plt.show()


### 3.3. Creación y evaluación de otro modelo (DecisionTree)

Veamos ahora otro modelo de predicción, un árbol de decisión (*decision tree*).

#### 3.3.1. Creación del modelo de árbol de decisión

Para esta tarea, tenemos la clase **DecisionTreeRegressor** del Scikit-learn.

In [ ]:

from sklearn import tree
model_tree = tree.DecisionTreeRegressor(random_state=SEED)
model_tree.fit(training_inputs, training_outputs)


#### 3.3.2. Evaluación con el dataset de entrenamiento

Una vez entrenado el modelo con el conjunto de entrenamiento, podemos ver qué tal es su desempeño sobre el conjunto de entrenamiento (midiéndolo con la métrica RMSE). El grado de acierto sobre las primeras muestras, y sobre todo el conjunto de entrenamiento, es:


In [ ]:

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("Training score:", model_tree.score(training_inputs, training_outputs))

dt_training_predictions = model_tree.predict(training_inputs)

print("Primeras 5 observaciones del entrenamiento")
for pred,real in zip(dt_training_predictions[:5].round(-2),training_outputs.iloc[:5].values):
    print(f"Valor real: {real}, valor predicho: {pred}. Diferencia en absoluto{abs(real-pred)}. Porcentaje desvío: {(abs(real-pred)*100/real).round(2)}%")

dt_rmse_train = root_mean_squared_error(training_outputs, dt_training_predictions)
print(f"RMSE del conjunto de entrenamiento con modelo de árbol de decisión: {dt_rmse_train:.2f}")

plt.scatter(training_outputs, dt_training_predictions)
plt.xlabel("Realidad")
plt.ylabel("Predicción")
plt.title("Árbol de decisión binario - Conjunto de entrenamiento")
plt.show()



**¿Qué ha pasado? ¿Cómo puede ser que no tenga ningún error?**

**EJERCICIO 1 PARA ENTREGAR EN EL AULA VIRTUAL**

Explica en una celda de texto qué puede estar pasando para que no exista error



In [ ]:
# EJERCICIO 1
# Explica en esta celda de texto que puede estar pasando para que no exista error





#### 3.3.3. Evaluación con un conjunto de datos no visto previamente

Veamos ahora cómo se comporta el modelo con un conjunto de datos que no ha visto durante el entrenamiento, por ejemplo, el conjunto de validación. La gráfica os mostrará la gran diferencia entre probar el modelo con datos usados en el entrenamiento, y con datos nunca vistos.

In [ ]:

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("Val score:", model_tree.score(val_inputs, val_outputs))

dt_val_predictions = model_tree.predict(val_inputs)

dt_rmse_val = root_mean_squared_error(val_outputs, dt_val_predictions)
print(f"RMSE del conjunto de validación con el modelo árbol de decisión: {dt_rmse_val:.2f}")

plt.scatter(val_outputs, dt_val_predictions)
plt.xlabel("Realidad")
plt.ylabel("Predicción")
plt.title("Árbol de decisión binario - Conjunto de validación")
plt.show()




#### 3.3.4. Visualización del árbol
Una de las ventajas de los modelos basados en árboles es su interpretabilidad. Scikit-learn permite visualizar los árboles de decisión. En el ejemplo siguiente podéis comprobar cómo se puede hacer. Debéis tener en cuenta que el árbol generado tiene una gran profundidad, por lo que, para poder visualizar algo, el ejemplo solo muestra los primeros 2 niveles (parámetro *manx_depth*). 

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(model_tree,
                   feature_names=training_inputs.columns.values,
                   max_depth=2,
                   filled=True)


### 3.4. Creación y evaluación de otro modelo (*Random Forest*)

Veamos ahora otro modelo de predicción, uno llamado *Random Forest*, que es un *ensemble* (conjunto) de *decision trees*. Cada uno se entrena con un conjunto aleatorio de los datos y de atributos. Al final, se promedia el resultado entre todos los árboles.

Para esta tarea, tenemos la clase [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) del Scikit-learn.

In [ ]:
try:
    import numpy as np
except ImportError as err:
    !pip install numpy
    import numpy as np
from sklearn.ensemble import RandomForestRegressor

#n_estimators es el número de árboles de decisión que componen el ensemble    
model_forest = RandomForestRegressor(n_estimators=80, random_state=SEED)

# ravel() convierte el segundo parámetro en un array 1D, lo que necesita su fit, DecisionTreeRegressor acepta ambos (1D o 2D)
#Esto es necesario realizarlo porque no todos los métodos de Scikit-learn están adaptados para trabajar con DataFrames de Pandas
model_forest.fit(training_inputs, np.array(training_outputs).ravel()) 
rf_training_predictions = model_forest.predict(training_inputs)

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("Training score:", model_forest.score(training_inputs, training_outputs))

print("Primeras 5 observaciones del entrenamiento")
for pred,real in zip(rf_training_predictions[:5].round(-2),training_outputs.iloc[:5].values):
    print(f"Valor real: {real}, valor predicho: {pred}. Diferencia en absoluto{abs(real-pred)}. Porcentaje desvío: {(abs(real-pred)*100/real).round(2)}%")

rf_rmse_train = root_mean_squared_error(training_outputs, rf_training_predictions)
print(f"RMSE del conjunto de entrenamiento con modelo de random forest: {rf_rmse_train:.2f}")

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("\nVal score:", model_forest.score(val_inputs, val_outputs))

rf_val_predictions = model_forest.predict(val_inputs)
rf_rmse_val = root_mean_squared_error(val_outputs, rf_val_predictions)
print(f"RMSE del conjunto de validación con el modelo random forest: {rf_rmse_val:.2f}")

plt.scatter(val_outputs, rf_val_predictions)
plt.xlabel("Realidad")
plt.ylabel("Predicción")
plt.title("Random forest - Conjunto de validación")
plt.show()


**Conclusión:** Como vemos, el modelo *Random Forest* da mejores resultados sobre el conjunto de validación.

Al comparar los modelos sobre un conjunto de validación, podríamos seleccionar el que obtiene mejores métricas y evaluarlo contra el conjunto de Test para obtener la métrica general. **Importante**: solo podríamos evaluarlo contra el conjunto de Test si no fuésemos a comparar o probar más configuraciones. Mientras no hayamos tomado todas las decisiones sobre el modelo, el conjunto de Test seguirá reservado.

In [ ]:
print("Resultados de validación (valores inferiores son mejores): ")
print(f"RSME LR = { lr_rmse_val:.2f}.  RSME DT = {dt_rmse_val:.2f}.     RSME RF = {rf_rmse_val:.2f}")



## 4. Validación cruzada (*Cross Validation*)


La validación cruzada es una técnica que nos permite dividir un conjunto en varias partes (*folds*), por ejemplo K, y realizar K entrenamientos con sus correspondientes evaluaciones. Cada uno de los entrenamientos emplea todas las particiones menos una (K-1), y emplea la parte restante para evaluar. En cada entrenamiento se cambia la partición empleada para evaluar. Finalmente se estudia la lista de resultados (evaluaciones), y se asigna un resultado del modelo de alguna forma (por ejemplo, haciendo la media).

El algoritmo se puede descomponer en los siguientes pasos:

1. Se divide el conjunto en ‘K’ partes (*folds*) iguales.
2. Se reserva una de las partes para evaluar.
3. Las otras K-1 partes se usan para entrenar.
4. Se evalúa el rendimiento del modelo en el *fold* reservado.
5. Se repiten los pasos 2, 3 y 4 ‘K’ veces variando en cada iteración el
*fold* de evaluación.


La validación cruzada nos permite *simular* dos conjuntos cuando solo tenemos uno. Se aplica habitualmente cuando el número de observaciones es pequeño y no tenemos un dataset suficientemente grande como para dividirlo en subconjuntos suficientemente significativos.

Si solo tenemos un dataset y aplicamos validación cruzada, podemos emplear el mismo dataset para entrenar y para testear. Si tenemos 2 subconjuntos (entrenamiento y test), podemos emplear la validación cruzada con el entrenamiento para entrenar y validar.

Por supuesto, es posible hacerlo a mano, pero, afortunadamente, Scikit-learn nos ofrece herramientas para hacerlo de forma automática (la función "[cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)"). Es importante también destacar que muchas de las funciones de preprocesado y optimización de Scikit-learn tienen la posibilidad de evaluarse a través de la validación cruzada.

Veamos ahora un ejemplo de cómo probar un modelo de los entrenados anteriormente con validación cruzada empleando [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html):



In [ ]:
from sklearn.model_selection import cross_val_score

# Empleamos validación cruzada con la regresión lineal. 5 folds
scores = cross_val_score(model_lin_reg, training_inputs, training_outputs, cv=5, scoring='neg_root_mean_squared_error')
# Según la documentación: All scorer objects follow the convention that higher return values are better than lower return values.
# Esa es la razón de que el RMSE venga en negativo. Lo pasamos a positivo para la correcta interpretación
scores*=-1


print("Resultados de la validación cruzada aleatoria estratificada para cada fold:")
for i, resultado in enumerate(scores):
    print(f"Fold {i}:{resultado:.2f}")

print(f"Media={scores.mean():.2f}" )

En el ejemplo anterior elegimos realizar una validación cruzada con 5 particiones (*folds*), pero, por defecto, las particiones se realizan siguiendo el orden secuencial de las observaciones, sin aleatoriedad ni estratificación. Si queremos hacerlo de forma aleatoria en el atributo de salida, podemos emplear la clase [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) para crear un objeto y dárselo al parámetro *cv*. Y si queremos hacerlo de forma estratificada sobre la variable a predecir (sólo aplicable en caso de ser una variable categórica), emplearemos la clase [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html).

In [ ]:
from sklearn.model_selection import KFold

skf = KFold(n_splits=5, shuffle=True, random_state=SEED)
scores = cross_val_score(model_lin_reg, training_inputs, training_outputs, cv=skf, scoring='neg_root_mean_squared_error')
scores*=-1

print("Resultados de la validación cruzada aleatoria estratificada para cada fold:")
for i, resultado in enumerate(scores):
    print(f"Fold {i}:{resultado:.2f}")

print(f"Media={scores.mean():.2f}" )


**Nota:** Hasta ahora, simplemente hemos entrenado el modelo directamente, sin intentar optimizarlo. Existen diferentes técnicas que podemos emplear para intentar mejorar las métricas del modelo, entre ellas podemos destacar:
* Técnicas para la reducción de la dimensionalidad.
* Técnicas para la optimización de los hiperparámetros de los modelos.

## 5. Reducción de la dimensionalidad



Trabajar con muchos atributos (dimensiones) puede parecer lo adecuado cuando tenemos que generar modelos y buscamos un alto rendimiento. Pero la realidad es que, cuando aumenta la dimensionalidad, el volumen del espacio de características aumenta exponencialmente, haciendo que los datos disponibles estén muy dispersos, lo que dificultará el trabajo de los modelos y que estos converjan y, como resultado, su rendimiento no será bueno. Es la llamada **maldición de la dimensionalidad**. 

El volumen y variabilidad de datos que necesitamos para entrenar en entornos de alta dimensionalidad es muy alto, y esto no siempre es fácil de obtener, por lo que, habitualmente, los modelos se ven beneficiados si reducimos la dimensionalidad. 

### 5.1. Selección de características

Una posibilidad para reducir la dimensionalidad es aplicar técnicas de selección de las características (*Feature Selection*, FS), es decir, eliminar las características que proporcionen poca o ninguna información al modelo y escoger las más relevantes. La aplicación de estas técnicas tiene como consecuencia directa reducir las dimensiones, e, indirectamente, nos proporciona información sobre el problema con el que estamos trabajando (nos muestra las variables más relevantes). Las técnicas de selección de características se engloban en 3 grandes grupos: Filtros, Embebidos y Wrappers.

**Nota**: *encontraréis una explicación más detallada de este apartado en la teoría de la materia*.

#### 5.1.1. Filtros
Técnicas de preprocesado que crean listas ordenadas de las características en base a algún tipo de métrica. El objetivo es eliminar las características con peores posiciones en el ranking (es necesario establecer un umbral).
* Son métodos simples y rápidos.
* Son robustos al sobreentrenamiento.
* Son fácilmente interpretables.
* Asignar el umbral no es sencillo.
* No valoran la combinación de variables (puede que dos variables independientes no aporten información al modelo, pero combinadas sí).
* Variables altamente correladas y redundantes no se eliminan (si dos variables tienen una muy alta correlación entre ellas, es decir, representan lo mismo, es probable que no tenga sentido mantener las dos).


Scikit-learn proporcina varios métodos de filtrado para poder aplicar sobre nuestros datos. Dos posibles estrategias una vez aplicado el método es:
1. Seleccionar las 'k' características mejor posicionadas en el ranking.
2. Seleccionar un porcentaje de las mejores características según el orden del ranking.

Dentro de los métodos de filtrado, uno de los más comunes es el filtrado a través de la correlación de Pearson. Veamos un ejemplo:

In [ ]:
## Ejemplos con filtros empleando Scikit-learn
from sklearn.feature_selection import SelectKBest, r_regression # r_regression es una función que mide la correlación Pearson

def plot_selected_features(features, scores, title=None, err=None):
    plt.figure(figsize=(20,10))
    plt.xticks(rotation=90)
    plt.xlabel("Características")
    plt.ylabel("Importancia")
    plt.title(title)
    plt.bar(features,scores,orientation='vertical', yerr=err )
    plt.show()


# Seleccionamos las 'K' mejores características
K=10

# creamos un K-selector que usa la correlación Pearson para ordenar las características
f_selector = SelectKBest(r_regression, k=K)

# calculamos la correlación entre las variables de entrada y la salida, y seleccionamos las de mayor correlación Pearson
# Necesitamos convertir el dataframe "training_outputs" en un array de 1D. training_outputs.values.ravel()
# training_outputs.values devuelve los valores en forma de array multidimensional (2D)
# training_outputs.values.ravel() "aplana" el array (lo convierte a 1D)
f_selector.fit(training_inputs, training_outputs.values.ravel())


print("Todas las características:\n"+"\n".join(training_inputs.columns.values))

indices_fs = f_selector.get_support(indices=True)
print(f"Índices de las {K} características seleccionadas:", indices_fs, "\n")

selected_fs = f_selector.get_feature_names_out(training_inputs.columns)
print(f"Nombres de las {K} características seleccionadas:\n"+ "\n".join(selected_fs))

scores = f_selector.scores_
print("Todos los scores:\n", scores, "\n")

selected_fs_tuplas = [(training_inputs.columns[i], scores[i]) for i in indices_fs]

print(f"Las {K} características con mayor correlación son:")
for t in sorted(selected_fs_tuplas, key = lambda t: t[1], reverse=True):  # ordenamos por score
    print(f"{t[0]}= {t[1]:.3f}")

# dibujamos los scores obtenidos para cada característica de entrada en su correlación con la salida
plot_selected_features(training_inputs.columns.values, f_selector.scores_, title="FS. Filtro correlación Pearson")


**Fijaos** que en el ejemplo anterior quizás no estemos seleccionando las que nos interesan, porque solo estamos **seleccionando las de mayor correlación positiva**. La correlación Pearson también nos da información interesante con la correlación negativa. Podemos modificar el código anterior para que tenga en cuenta el valor absoluto. Comprobaréis en el resultado que nos selecciona otras distintas.

In [ ]:
def abs_r_regression(inputs, outputs):
    return np.abs(r_regression(inputs, outputs))  

# Seleccionamos las 'K' mejores características
K=10

# creamos un K-selector que usa el valor absoluto de la correlación Pearson 
f_selector = SelectKBest(abs_r_regression, k=K)

# calculamos la correlación entre las variables de entrada y la salida, y seleccionamos las de mayor correlación Pearson
f_selector.fit(training_inputs, training_outputs.values.ravel())

#get_support(indices=True) devuelve los índices de las características seleccionadas
indices_fs = f_selector.get_support(indices=True)
#get_feature_names_out devuelve los nombres las características seleccionadas
selected_fs = f_selector.get_feature_names_out(training_inputs.columns)
#scores_ devuelve la puntuación de cada característica
scores = f_selector.scores_

selected_fs_tuplas = [(training_inputs.columns[i], scores[i]) for i in indices_fs]

print(f"Las {K} características con mayor correlación absoluta son:")
for t in sorted(selected_fs_tuplas, key = lambda t: t[1], reverse=True):
    print(f"{t[0]}= {t[1]:.3f}")

# dubujamos los scores obtenidos por cada característica de entrada en su correlación con la salida
plot_selected_features(training_inputs.columns.values, f_selector.scores_, title="FS. Filtro correlación Pearson (ABS)")



#### 5.1.2. Embebidos
Los métodos de selección de características embebidos incorporan la selección de características en el propio proceso de entrenamiento de un modelo de ML. Hay dos tipos principales:
1. **Aproximaciones basadas en regularización** (ej. Lasso-L1). Son algoritmos que tratan de minimizar una función de coste en la que participan términos de regularización. En el proceso de construcción del modelo se eliminan las características cuyo peso sea 0 ($\beta_j = 0$).

$$\large F_{coste}=\sum_{i=1}^N(y_i-\sum(x_{ij}\beta_j))^2 + \lambda\sum_{j=1}^P|\beta_j|$$
$$\large L_1=\lambda\sum_{j=1}^P|\beta_j|$$ 


**Importante**: con este método solo podríamos eliminar las características que sean 0. Aumentar el parámetro $\lambda$ tiene como consecuencia que el modelo intente dar menos importancia a cada una de las características y, en consecuencia, puede asignar 0 a algunas de estas.

En el ejemplo siguiente podéis jugar con el parámetro *alpha* que representa a $\lambda$ en la ecuación. Si ponéis un valor igual a 1, *eliminará* alguna característica.

Entrenando el modelo podéis analizar los coeficientes asociados a cada característica para poder descartar los que son 0, pero Scikit-learn proporciona una clase ([SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html)) que nos permite seleccionar las características relevantes de forma automática.
       

In [ ]:
## Ejemplo embebido basado en regularización

from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model

# Modelo de regularización

lasso_model = linear_model.Lasso(alpha=0.1, max_iter=1000, random_state=SEED)
f_selector = SelectFromModel(estimator=lasso_model)
f_selector.fit(training_inputs,training_outputs)
selected_fs = f_selector.get_feature_names_out(training_inputs.columns)

print(f"Características originales: {len(training_inputs.columns)}. \n Características seleccionadas: {len(selected_fs)}")

print("Características seleccionadas por el método de regularización: \n"+" \n".join(selected_fs))


plot_selected_features(training_inputs.columns.values, f_selector.estimator_.coef_, title="FS. Embebido modelo con regularización")


       
2. **Aproximaciones basadas en la información extraída con la construcción del modelo**. El ejemplo más claro son los **modelos
basados en árboles**, en los que la propia construcción del árbol otorga la importancia a cada de las características:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

# n_estimators es el número de árboles en el ensemble
model = RandomForestRegressor(n_estimators=50)

# training_outputs.values.ravel() no todos los métodos soportan un DataFrame
model.fit(training_inputs, training_outputs.values.ravel())

importances = model.feature_importances_ # importancia general de los atributos en el RandomForest
std = np.std([importances for tree in model.estimators_],axis=0) # Podemos calcular la importancia para cada árbol y con ello la std

# argsort devuelve analiza un array devuelve un array con los índices (posición de los elementos) ordenados de menor a mayor
# [::-1] invierte un array
print("Características ordenadas de mayor a menor importancia")
for idx in np.argsort(importances)[::-1]:
    print(f"{training_inputs.columns[idx]}: {importances[idx]:.3f}")

plot_selected_features(training_inputs.columns.values, importances, title="FS. Embebido RandomForest")



#### 5.1.3. Wrappers

Los *wrappers* son algoritmos de selección de características que integran un predictor durante el proceso de selección. Se prueban diferentes combinaciones de atributos para entrenar un modelo, y el subconjunto con mejores resultados es el seleccionado. Es inviable computacionalmente probar todas las posibles combinaciones, por lo que es necesario establecer un método de búsqueda en el espacio de características que genere los subconjuntos.
* Son simples, universales, tienen buen rendimiento (el modelo resultante), PERO:
    * Son computacionalmente muy costosos.
    * Dependen en gran medida del modelo seleccionado.
    * Pueden caer en el sobreentrenamiento.
    * Es difícil configurar los modelos de forma justa (agregar el proceso de optimización de hiperparámetros produciría una sobrecarga muy importante).
    * Cada método para seleccionar subconjuntos tiene problemáticas asociadas que es necesario valorar.

Dos de los métodos más sencillos para establecer los conjuntos son:
1. **Sequential forward selection (SFS) algorithm**:
    1. Empezamos con un conjunto vacío FS_DATASET.
    2. Añadimos a FS_DATASET la característica X que maximiza el rendimiento del modelo.
    3. Repetimos el paso 'B' hasta que se cumple la condición de salida. 


In [ ]:
## Ejemplos wrappers

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn import tree

# Features to select
K=10
model = tree.DecisionTreeRegressor()#modelo que usaremos para evaluar la bondad del subconjunto de características

sfs = SequentialFeatureSelector(model, n_features_to_select=K, direction='forward')

sfs.fit(training_inputs, training_outputs)
selected_features = sfs.get_feature_names_out(training_inputs.columns)
print("Características seleccionadas empleando SFS:\n" + "\n".join(selected_features))


2. **Sequential backward selection (SBS) algorithm**:
    1. Empezamos con un conjunto completo FS_DATASET (con todas las posibles características).
    2. Eliminamos de FS_DATASET la característica X que menos reduce el rendimiento del predictor (la más irrelevante).
    3. Repetimos el paso 'B' hasta que se cumple la condición de salida.

In [ ]:
## Ejemplos wrappers

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn import tree

# Features to select
K=10
model = tree.DecisionTreeRegressor()#modelo que usaremos para evaluar la bondad del subconjunto de características

sfs = SequentialFeatureSelector(model, n_features_to_select=K, direction='backward')

sfs.fit(training_inputs, training_outputs)
selected_features = sfs.get_feature_names_out(training_inputs.columns)
print("Características seleccionadas empleando SBS:\n" + "\n".join(selected_features))


### 5.2. Reducción de dimensionalidad a través de proyecciones

Este tipo de métodos generan un nuevo y más pequeño conjunto características (reduciendo la dimensionalidad), cada una de las cuales es una combinación de las variables de entrada. **No generan una selección de características**, sino que se realiza una transformación.

Uno de los métodos más conocidos es el análisis de componentes principales.

#### 5.2.1. Principal Component Analisys (PCA)
El PCA permite reducir la dimensionalidad (características) perdiendo la menor cantidad de información (varianza). El objetivo es reducir un número elevado de características, posiblemente correlacionadas (información redundante), a un número menor de variables transformadas (componentes principales) que explique gran parte de la variabilidad de los datos. 
* Cada componente principal será una combinación lineal de las variables originales.
* Las componentes principales serán independientes (no correlacionadas entre sí).
* Los componentes se ordenan por la cantidad de varianza original que describen.
* Podemos seleccionar las ‘n’ componentes principales que explican un alto porcentaje de varianza (ej. 95%) y, con ello, reducir la dimensionalidad.

Scikit-learn tiene la clase [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) que permite aplicar este método a nuestros datos:


In [ ]:
from sklearn.decomposition import PCA

# n_components, si ponemos un valor entero 'x', seleccionará las 'x' componentes principales
# n_components, si ponemos un valor flotante 'y' entre 0 y 1 , seleccionará las componentes principales que expliquen el 'y'% de la varianza
porcentaje_varianza = 0.99
pca = PCA(n_components=porcentaje_varianza)
training_inputs_pca = pca.fit_transform(training_inputs)

print(f"Número de características originales: {len(training_inputs.columns)}")
print(f"Número de componentes principales seleccionadas: {training_inputs_pca.shape[1]}")
print(f"Se redujo la dimensionalidad en {len(training_inputs.columns)-training_inputs_pca.shape[1]} dimensiones")

print("\nVarianza explicada por cada una de las componentes principales: ")
varianza_acumulada=0
for i, valor in enumerate(pca.explained_variance_ratio_, start=1):
    varianza_acumulada+=valor
    print(f"Componente principal {i}: {valor:.3f}. Varianza acumulada: {varianza_acumulada:.3f}")


# Las componentes principales son una combinación lineal de las características originales. 
# Veamos como se forma la componente principal 1
first_pc = pca.components_[0] 
equation = ")+(".join([f"{coef:.4f} * {name}" for coef, name in zip(first_pc, training_inputs.columns.values)])
print(f"\nLa primera componente principal se forma como:\nPC1 = ({equation})")





### 5.3. Seleccionar las características para nuestro modelo

Tanto los métodos de selección de características, como los de reducción de la dimensionalidad a través de proyecciones (ej. PCA) nos proporcionan conjuntos de datos con menor dimensionalidad. Cada uno de ellos genera una respuesta diferente. **¿Cuál escoger?** A priori, no existe una respuesta correcta, es decir, la única opción sería probar los resultados de cada uno de estos métodos con el conjunto de validación, es decir, entrenar el modelo con cada una de estas alternativas (ej. el subconjunto de características generado por el filtro de correlación), y evaluar el modelo resultante contra el conjunto de validación. Una vez hayamos evaluado todas las alternativas, podríamos comparar las métricas resultantes y seleccionar el modelo (con la configuración asociada) que tuvo mejores valores.

El siguiente ejemplo muestra cómo entrenar el modelo con el resultado del PCA, y cómo evaluarlo con el conjunto de validación:






In [ ]:
   
model_forest = RandomForestRegressor(n_estimators=80, random_state=SEED)
# ravel() convierte el segundo parámetro en un array 1D, lo que necesita su fit, DecisionTreeRegressor acepta ambos (1D o 2D)
model_forest.fit(training_inputs_pca, np.array(training_outputs).ravel()) 

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("Training score:", model_forest.score(training_inputs_pca, training_outputs))

rf_training_predictions_pca = model_forest.predict(training_inputs_pca)

print("Primeras 5 observaciones del entrenamiento")
for pred, real in zip(rf_training_predictions_pca[:5].round(-2), training_outputs.iloc[:5].values):
    print(f"Valor real: {real}, valor predicho: {pred}. Diferencia en absoluto{abs(real-pred)}. Porcentaje desvío: {(abs(real-pred)*100/real).round(2)}%")

rf_rmse_train = root_mean_squared_error(training_outputs, rf_training_predictions_pca)
print(f"RMSE del conjunto de entrenamiento con modelo de random forest: {rf_rmse_train:.2f}")

val_inputs_pca = pca.transform(val_inputs)

# La propia clase nos da un score para el modelo, pasándole las entradas y salidas
print("\nVal score:", model_forest.score(val_inputs_pca, val_outputs))

rf_val_predictions_pca = model_forest.predict(val_inputs_pca)
rf_rmse_val_pca = root_mean_squared_error(val_outputs, rf_val_predictions_pca)
print(f"RMSE del conjunto de validación con el modelo random forest CON PCA: {rf_rmse_val_pca:.2f}")

plt.scatter(val_outputs, rf_val_predictions_pca)
plt.xlabel("Realidad")
plt.ylabel("Predicción")
plt.title("Random forest - Conjunto de validación")
plt.show()



En este caso concreto el uso del PCA no parece mejorar el entrenamiento respecto a emplear todas las características. Habría que probar también con el resultado de las diferentes técnicas de selección de características vistas anteriormente y compararlas. 

**Importante**: Las técnicas de reducción de dimensionalidad funcionan mejor si tenemos una gran número de dimensiones en las que existen características poco o nada relevantes o existen características redundantes. Nuestro problema de ejemplo no tiene una gran dimensionalidad y por ello no estará demasiado afectado por la *maldición de la dimensionalidad*.

In [ ]:
print(f"RMSE del conjunto de validación con el modelo random forest CON PCA: {rf_rmse_val_pca:.2f}")
print(f"RMSE del conjunto de validación con el modelo random forest SIN usar PCA: {rf_rmse_val:.2f}")

**EJERCICIO 2 PARA ENTREGAR EN EL AULA VIRTUAL**

Utiliza el subconjunto de características generado por alguno de los métodos de selección de características vistos anteriormente (filtros, embebidos o wrappers) para entrenar un modelo y validarlo con el conjunto de validación. Compara las métricas con la salida del PCA, y con no aplicar la reducción de dimensionalidad.



In [ ]:
# EJERCICIO 2
# Utiliza el subconjunto de características generado por alguno de los métodos de selección de características 
# vistos anteriormente (filtros, embebidos o wrappers) para entrenar un modelo y validarlo con el conjunto de validación. 
# Compara las métricas con la salida del PCA y con no aplicar la reducción de dimensionalidad.



## 6. Optimización de hiperparámetros
Los algoritmos de ML tienen una serie de parámetros de configuración que afectan al entrenamiento del modelo y posteriormente al desempeño de éste (ej. *learning rate* en regresión lineal, número de estimadores en *Random Forest*, etc.). Los parámetros por defecto raras veces son los óptimos, y es necesario optimizarlos para cada problema concreto. Este es un proceso muy costoso computacionalmente, y que requiere muchas veces de recursos dedicados. Existen diferentes técnicas para realizar la optimización (ej. GRID, aleatoria, evolutiva, etc.). La más básica es la optimización basada en una búsqueda GRID.

### 6.1. Grid Search

La optimización GRID es la prueba exhaustiva de todas las combinaciones posibles de varios parámetros (Scikit-learn nos proporciona la clase **GridSearchCV** para esta tarea). Para aplicarla, lo primero que necesitamos es establecer los rangos de valores que queremos probar. La prueba exhaustiva es el equivalente a probar todas las combinaciones empleando bucles “for” anidados. Las evaluaciones de hiperparámetros se hacen y se comparan a través de un conjunto de validación (o empleando validación cruzada), pero nunca contra el conjunto de Test).

El siguiente es un ejemplo de GridSearchCV sobre el modelo de árbol de decisión, donde exploramos:
- Dos valores para el hiperparámetro *criterion*, empleado para decidir la forma de creación de nodos al constuir el árbol.
- Cuatro valores para *max_features*, donde se indican los porcentajes de atributos que se incluirán en la creación del árbol (los atributos se eligen al azar).


Emplearemos la validación cruzada con los datos de entrenamiento para seleccionar los hiperparámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV

# estos son los parámetros a analizar y los valores de cada uno a explorar
param_grid = {
    'criterion': ["squared_error", "absolute_error"], # para decidir la creación de nodos al constuir el árbol
    'max_features': [0.25, 0.5, 0.75, 1.0]  # porcentajes de las features incluidas
}


# buscaremos los mejores parámetros para el modelo de árbol de decisión (model_tree)
# cv puede ser un número fijo, un KFold, o una lista de tuplas (train,test) de índices POSICIONALES a probar 
# Por simplificación emplaremos un CV=5
grid_search = GridSearchCV(estimator=model_tree, param_grid=param_grid, 
                           scoring="neg_root_mean_squared_error", # criterio para seleccionar la mejor combinación de parámetros
                           return_train_score=True,
                           cv=5)



results = grid_search.fit(training_inputs, training_outputs)

print(f"Mejor score en validación: {-grid_search.best_score_:.2f}")
print(f"Params del mejor score en validación: {grid_search.best_params_}")


Y, a continuación, otro ejemplo de GridSearchCV con el modelo *Random Forest*, donde exploramos:
- 4 valores para el hiperparámetro *max_features*, el número de atributos usados al construir cada árbol del modelo.
- 5 para *n_estimators*, el número de árboles usados en el modelo.


In [ ]:
# ahora con Random Forest
# estos son los parámetros a analizar y los valores de cada uno a explorar
param_grid = {
    'max_features': [2,4,6,8],
    'n_estimators': [70,80,90,100,110]  
}

# buscaremos los mejores parámetros para el modelo de árbol de decisión (model_tree)
# cv puede ser un número fijo, un KFold, o una lista de tuplas (train,test) de índices POSICIONALES a probar 
grid_search = GridSearchCV(estimator=model_forest, param_grid=param_grid, 
                           scoring="neg_root_mean_squared_error", # criterio para seleccionar la mejor combinación de parámetros
                           return_train_score=True,
                           cv=5)


results = grid_search.fit(training_inputs, np.array(training_outputs).ravel())

print(f"Mejor score en validación: {-grid_search.best_score_}")
print(f"Params del mejor score en validación: {grid_search.best_params_}\n")



El mejor resultado puede variar de ejecución en ejecución, ya que hay cierta aleatoriedad en los conjuntos que procesa cada árbol.

Si dibujásemos los resultados (información contenida en *results* en el anterior código), veríamos algo como la siguiente figura, que muestra que la evaluación se realiza para todas las combinaciones, y las zonas donde se producen mejores y peores modelos (la figura no tiene relación con los datos, es sólo un ejemplo).


| ![alt text](img/gridSearch.png "Grid Search") | 
|:--:| 
| **Grid Search**: Búsqueda a través de diferentes valores de dos hiperparámetros. Para cada hiperparámetro se consideran 10 valores diferentes (100 combinaciones distintas). Los contornos azules indican las regiones con mejores resultados, mientras que los rojos son regiones con peores resultados. Fuente de la imagen [wikipedia](https://es.m.wikipedia.org/wiki/Archivo:Hyperparameter_Optimization_using_Grid_Search.svg)|

### 6.2. Random Search


En este caso, realizamos una búsqueda aleatoria (no completa) en el espacio de combinaciones de los diferentes valores de los hiperparámetros. Con una búsqueda aleatoria suficientemente grande, esta aproximación tiene como ventajas:
* Explora un espacio de búsqueda más amplio.
* Es menos costoso computacionalmente.
* Empiricamente [[5]](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf?ref=broutonlab.com) ha demostrado ser más eficiente que la búsqueda exhaustiva.



In [ ]:
# Ejemlo de Random Search con Random Forest

from sklearn.model_selection import RandomizedSearchCV

# estos son los parámetros a analizar y los valores de cada uno a explorar
param_distributions = {
    'max_features': [2,4,6,8],
    'n_estimators': [70,80,90,100,110]  
}

# buscaremos los mejores parámetros para el modelo de árbol de decisión (model_tree)
# cv puede ser un número fijo, un KFold, o una lista de tuplas (train,test) de índices POSICIONALES a probar (este caso)
# en este caso, cv será una lista de un único elemento, una tupla que recoge todos los índices POSICIONALES de entrenamiento y validación
random_search = RandomizedSearchCV(estimator=model_forest, param_distributions=param_distributions,
                           scoring="neg_root_mean_squared_error", # criterio para seleccionar la mejor combinación de parámetros
                           return_train_score=True, n_iter=10, # límite de combinaciones a probar (10 es el valor por defecto) 
                           cv=[(np.arange(len(training_inputs)),  # índices para entrenamiento
                                np.arange(len(training_inputs),len(training_inputs) + len(val_inputs)) # índices para validación
                               )])

# x e y tienen los índices originales del dataframe en su columna idx, pero esto no se usará por el GridSearchCV.
# si el primer índice del cv-train es '0', GridSearchCV cogerá el valor de la primera posición de x, no el índice idx=0.
x = pd.concat([training_inputs, val_inputs])
y = pd.concat([training_outputs, val_outputs])

results = random_search.fit(x, np.array(y).ravel())

print(f"Mejor score en validación: {-random_search.best_score_}")
print(f"Params del mejor score en validación: {random_search.best_params_}")


Limitando nuestra búsqueda a solo  10 combinaciones, es posible que el resultado óptimo no sea el encontrado pero podríamos aumentar el número de iteraciones para que el resultado final sea más confiable.

Si repitiésemos la figura anterior para el caso del Random Search, veríamos que las combinaciones probadas muestran una forma aleatoria, no la rejilla anterior.

| ![alt text](img/randomSearch.png "Random Search")| 
|:--:| 
| **Random Search**: Búsqueda aleatoria entre diferentes combinaciones de valores para dos hiperparámetros. En este ejemplo se evalúan 100 opciones aleatorias diferentes.Los contornos azules indican las regiones con mejores resultados, mientras que los rojos son regiones con peores resultados. Fuente de la imagen [wikipedia](https://commons.wikimedia.org/wiki/File:Hyperparameter_Optimization_using_Random_Search.svg)|


**EJERCICIO 3 PARA ENTREGAR EN EL AULA VIRTUAL**

En este punto nos queda unir todo lo que hemos visto con el objetivo de seleccionar un modelo y testearlo. En este ejercicio deberéis realizar:
1. Seleccionar 2 modelos:
    * Un Random Forest.
    * Un modelo de regresión presente en Scikit-learn que no se haya empleado en esta práctica (ANN, SVMs, etc.).
2. Reducir la dimensionalidad empleando PCA y desarrollar los modelos.
3. Optimizar los modelos con una búsqueda de Random Search empleando CV=5.
    * Como mínimo, es necesario optimizar 2 hiperparámetros.
4. Comparar los resultados de los modelos optimizados sobre el conjunto de validación.
5. Seleccionar el mejor modelo en base a sus métricas de validación y testearlo con el conjunto de Test.
6. Mostrar las métricas generales generadas por el Test.


**Nota 1**: Una parte de los apartados están ya realizados, sobre todo los relacionados con el Random Forest.  Fijaros que el ejemplo del Random Search no se está empleando PCA, por lo que tenéis que adaptarlo.


**Nota 2**: Todo el código necesario para el desarrollo del ejercicio debe aparecer en las celdas siguientes (incluso si algún apartado puede reaprovecharse de celdas anteriores). El objetivo es desarrollar el flujo de trabajo completo.

In [ ]:
# EJERCICIO 3
